Show git repo organization options?

Show router tree, module import tree, call tree, feature tree?

Warn of changes made outside of the module or feature branch?

Lint all code?  Reformat all code using webstorm?

Code analysis to combine blocks into modules?



TODO: use language server for this

generate unit test from logic branching?


In [ ]:
var acorn = require("acorn");
var ASTQ  = require("astq");
var _ = require('lodash');
var importer = require('../Core');

var source = importer.interpret('zuora eloqua mapper').code;
    
var comments = [], tokens = [];

var ast = acorn.parse(source, {
    ecmaVersion: 6,
    // collect ranges for each node
	ranges: true,
	// collect comments in Esprima's format
	onComment: comments,
	// collect token ranges
	onToken: tokens
})

var astq = new ASTQ();
var assignments = astq.query(ast, `
//AssignmentExpression [
    //Identifier[@name=='record']
]
`);
var grouped = _.groupBy(assignments, a => comments.filter(c => c.start < a.start).pop().value.trim());
var output = '';
Object.keys(grouped).forEach(k => {
    const ass = grouped[k].map(as => {
        const leftMembers = as.left.type === 'MemberExpression'
            ? [as.left.property.value] : astq.query(as.left, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        const rightMembers = as.right.type === 'MemberExpression'
            ? [as.right.property.value] : astq.query(as.right, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        if(leftMembers.length === 0 || rightMembers.length === 0) {
            return;
        }
        return `
            it('should map ${leftMembers.join(', ')} to eloqua given ${rightMembers.join(', ')} from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['${rightMembers.join('\'] || records[0][\'')}'], mapped[0]['${leftMembers}']);
            })
`;
    }).filter(m => m).join('\n');
    output += `
        describe('${k} properties mapped', () => {
            ${ass}
        });
`
})
//console.log(astq.query(ast, `//AssignmentExpression`));
var destination = [].concat(...astq.query(ast, `//AssignmentExpression`)
    .map(as => as.left.type === 'MemberExpression'
            ? [as.left.property.value]
            : astq.query(as.left, `//MemberExpression`)
         .map(m => m.property.value)
         )).filter(m => m)
var source = astq.query(ast, `
//MemberExpression [
    //Literal
]
`)
    .filter(e => (e.property.value + '').includes('.'))
    .map(e => e.property.value)
console.log(`
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        ${output}
    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = ${JSON.stringify(source, null, 4).replace(/\n/ig, '\n    ')};
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

`);


get ast path array?

In [ ]:
var ASTQ  = require("astq");
var astq = new ASTQ();
var esquery = require('esquery');
var esprima = require('esprima'); 

function getArrayAST(selector, ctx, index) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true, comment: true});
    }
    try {
        return astq.query(ctx, selector);
    } catch (e) {
        if(e.message.includes('query parsing failed')) {
            return esquery(ctx, selector);
        } else {
            throw e;
        }
    }
}

// TODO: implement getObjectXPath with ASTs

module.exports = getArrayAST;



test ast path array?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

function testASTArray(selector, search = '') {
    return getArrayAST(selector, importer.interpret(search).code);
}

module.exports = testASTArray;



get module exports from source?


In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

var GET_EXPORTS = `//AssignmentExpression[//Identifier[@name=='exports']]/Identifier`;
var GET_FUNCTIONS = `//FunctionDeclaration`;

// TODO: does this work on other node_modules?
function getExports(code) {
    // can't call it exports or it might detect itself
    var moduleOutput = getArrayAST(GET_EXPORTS, code)
        .map(e => e.name);
    return getArrayAST(GET_FUNCTIONS, code)
        .filter(f => moduleOutput.includes(f.id.name))
        .map(f => f.id.name);
}

module.exports = getExports;


get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');

function getParameterNames(search) {
    const result = typeof search === 'function'
        ? search.toString()
        : importer.interpret(search).code;
    
    var fnName = getExports(result)[0];
    var GET_PARAMETERS = `//FunctionDeclaration[
            /Identifier[@name == '${fnName}']
        ]/*[@type != 'BlockStatement']//Identifier`;
    var parameters = getArrayAST(GET_PARAMETERS, result).map(p => p.name);

    console.log(`Parameter names ${fnName} ${parameters}`);
    return [fnName, ...(parameters || [])];
}

module.exports = getParameterNames;


get imports from source?

In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import notebook.ipynb');
var getArrayAST = importer.import('get ast path array');
var GET_IMPORTS = `//CallExpression[//Identifier[@name=='import' || @name=='importNotebook']]//Literal`;
var GET_IMPORTER = `//CallExpression//Identifier[
    @name=='interpret' || @name=='runAllPromises' || @name=='runInNewContext' ||
    @name=='getCells' || @name=='streamJson' || @name=='regexToArray']`

function matchImports(code) {
    var imports = getArrayAST(GET_IMPORTS, code)
        .map(s => s.value);
    var coreMatches = getArrayAST(GET_IMPORTER, code)
        .map(s => s.name)
        .map(i => importerCode.filter(core => getExports(core.code).includes(i))[0].id);
    return imports.concat(coreMatches);
}

module.exports = matchImports;



get requires from source?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

var GET_REQUIRE = `//CallExpression[//Identifier[@name=='require']]//Literal`;

function getRequires(code) {
    var requires = getArrayAST(GET_REQUIRE, code)
        .map(s => s.value);
    return requires;
}


module.exports = getRequires;

